In [1]:
import json
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import re
import json
import pandas as pd
import math
import tensorflow as tf
from numpy.random import seed
import sys

In [12]:
reviews = []
with open('data/train.jsonl') as fl:
    for i, line in enumerate(fl):
        reviews.append(json.loads(line))
print(len(reviews))
df = pd.DataFrame(reviews)
print(df.shape)
df.head()


5000
(5000, 3)


,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [13]:
import re 
def function_clean(text):
    text = re.sub(r"http\S+", "", text) #removing the URL Http
    # Removal of mentions
    text = re.sub("@[^\s]*", "", text)
    # Removal of hashtags
    text = re.sub("#[^\s]*", "", text)
    # Removal of numbers
    text = re.sub('[0-9]*[+-:]*[0-9]+', '', text)
    text = re.sub("'s", "", text)
    
    return text

In [14]:
df["context"]= df["context"].str.join(" ") 
df['text'] = df['response'] + df['context']
df['text'] = df['text'].apply(lambda text: function_clean(text))

In [15]:
df['label'] = df['label'].apply(lambda x: np.where(df['label'].unique()== x )[0][0])
df['text'][0]

"   I don't get this .. obviously you do care or you would've moved right along .. instead you decided to care and troll her ..A minor child deserves privacy and should be kept out of politics . Pamela Karlan , you should be ashamed of your very angry and obviously biased public pandering , and using a child to do it .  If your child isn't named Barron ...  Melania couldn't care less . Fact . 💯"

In [16]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in df['text']]

[nltk_data] Downloading package punkt to /Users/shengyi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [21]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]
df['Text_Final'] = [' '.join(sen) for sen in filtered_words]
df['tokens'] = filtered_words

In [22]:
pos = []
neg = []
for l in df['label']:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)
df['Pos']= pos
df['Neg']= neg

data = df[['Text_Final', 'tokens', 'label', 'Pos', 'Neg']]
data.head()

,Text_Final,tokens,label,Pos,Neg
0,n't get .. obviously care would 've moved righ...,"[n't, get, .., obviously, care, would, 've, mo...",0,0,1
1,trying protest . talking labels label wtf make...,"[trying, protest, ., talking, labels, label, w...",0,0,1
2,"makes insane money movies , einstein ! j . tru...","[makes, insane, money, movies, ,, einstein, !,...",0,0,1
3,meanwhile trump wo n't even release sat scores...,"[meanwhile, trump, wo, n't, even, release, sat...",0,0,1
4,pretty sure anti-lincoln crowd claimed `` demo...,"[pretty, sure, anti-lincoln, crowd, claimed, `...",0,0,1


In [23]:
print(data['tokens'][0])

["n't", 'get', '..', 'obviously', 'care', 'would', "'ve", 'moved', 'right', 'along', '..', 'instead', 'decided', 'care', 'troll', '..a', 'minor', 'child', 'deserves', 'privacy', 'kept', 'politics', '.', 'pamela', 'karlan', ',', 'ashamed', 'angry', 'obviously', 'biased', 'public', 'pandering', ',', 'using', 'child', '.', 'child', "n't", 'named', 'barron', '...', 'melania', 'could', "n't", 'care', 'less', '.', 'fact', '.', '💯']


In [24]:
all_training_words = [word for tokens in data["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))


400109 words total, with a vocabulary size of 29184
Max sentence length is 829


In [25]:

from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

In [26]:
word2vec_path = '../GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [27]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)

In [15]:
training_embeddings = get_word2vec_embeddings(word2vec, data, generate_missing=True)

In [28]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [29]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 27781 unique tokens.


In [30]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [31]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(27782, 300)


In [32]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    convs = []
    filter_sizes = [2,3,4,5,6]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)


    l_merge = concatenate(convs, axis=1)

    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [33]:
label_names = ['Pos', 'Neg']

In [34]:
y_train = data[label_names].values

In [35]:
x_train = train_cnn_data
y_tr = y_train
print(y_tr)

[[0 1]
 [0 1]
 [0 1]
 ...
 [1 0]
 [1 0]
 [1 0]]


In [36]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 50, 300)      8334600     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 49, 200)      120200      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 48, 200)      180200      embedding[0][0]                  
_______________________________________________________________________________________

In [37]:
num_epochs = 3
batch_size = 80
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Epoch 1/3
57/57 [==============================] - 5s 82ms/step - loss: 0.6819 - acc: 0.6076 - val_loss: 0.5151 - val_acc: 0.7620
Epoch 2/3
57/57 [==============================] - 4s 69ms/step - loss: 0.5377 - acc: 0.7324 - val_loss: 0.9253 - val_acc: 0.4080
Epoch 3/3
57/57 [==============================] - 4s 68ms/step - loss: 0.4440 - acc: 0.8082 - val_loss: 1.1299 - val_acc: 0.3740


In [38]:
test = []
with open('data/test.jsonl') as fl2:
    for i, line in enumerate(fl2):
        test.append(json.loads(line))
print(len(test))
df2 = pd.DataFrame(test)
df2["context"]= df2["context"].str.join(" ") 
df2['text'] = df2['response'] + df2['context']
df2['text'] = df2['text'].apply(lambda text: function_clean(text))
tokens2 = [word_tokenize(sen) for sen in df2['text']]
lower_tokens2 = [lower_token(token) for token in tokens2]
filtered_words2 = [remove_stop_words(sen) for sen in lower_tokens2]
result2 = [' '.join(sen) for sen in filtered_words2]
print(len(result2))
df2['Text_Final'] = result2
df2['tokens'] = filtered_words2
test_sequences = tokenizer.texts_to_sequences(df2["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

1800
1800


In [39]:
predictions = model.predict(test_cnn_data, verbose=1)

57/57 [==============================] - 0s 8ms/step


In [40]:
labels = [1, 0]

In [56]:
prediction_labels=[]
for p in predictions:
    if p[1] > 0.38:
        prediction_labels.append(0)
    else:
        prediction_labels.append(1)
        

In [57]:
print(len(prediction_labels))

1800


In [58]:
df_pred = pd.DataFrame(data = prediction_labels, columns=["prediction"])

In [59]:
df_pred["twitter"]= (df_pred.index + 1)
df_pred["twitter_index"]= "twitter_" + df_pred["twitter"].astype(str)
df_pred["result"] = df_pred['prediction'].apply(lambda x: 'SARCASM' if x == 0 else 'NOT_SARCASM')

In [60]:
df_pred[["twitter_index", "result"]].to_csv("answer.txt", header=None, index=None, sep=',', mode='w')

In [61]:
print(predictions)

[[0.39070976 0.580865  ]
 [0.10693663 0.86743605]
 [0.24639711 0.72153306]
 ...
 [0.05243149 0.931767  ]
 [0.88335574 0.10696602]
 [0.9750471  0.02520457]]


In [62]:
print(prediction_labels)

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 